In [3]:
# Import des bibliothèques nécessaires
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [8]:
# Charger le jeu de données
path = "/home/onyxia/work/statapp_sujet26/"
file_name1="dataset_complet_avec_dummies_part_1.csv"
file_name2="dataset_complet_avec_dummies_part_2.csv"
df1= pd.read_csv(path+file_name1, sep=',',low_memory=False)
df2= pd.read_csv(path+file_name2, sep=',',low_memory=False)
df=pd.concat([df1,df2])


In [9]:
# Modifier la colonne 'grav' pour un problème de classification binaire
df['grav'] = df['grav'].replace({1: 0, 2: 0, 3: 1, 4: 1})

# Diviser le jeu de données en ensembles d'entraînement et de test
y = df['grav']
X = df.drop(columns=['grav'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [10]:
# Créer un classifieur Random Forest
rf_classifier = RandomForestClassifier(random_state=42)

# Entraîner le modèle
rf_classifier.fit(X_train, y_train)

# Extraire l'importance des features
importances = rf_classifier.feature_importances_

# Calculer le pourcentage d'importance de chaque feature
total_importance = np.sum(importances)
percentage_importance = (importances / total_importance) * 100

In [12]:
# Créer un DataFrame pour stocker les noms des features et leur pourcentage d'importance
feature_names = X.columns
indices = np.argsort(importances)[::-1]
feature_df = pd.DataFrame({'Feature': feature_names[indices], 'Importance (%)': percentage_importance})

print(feature_df)

                  Feature  Importance (%)
0                     vma        4.758959
1                     nbv        5.538648
2    region_Île-de-France        1.603772
3                catr_3.0        1.794340
4                 agg_2.0        1.647418
..                    ...             ...
104               int_8.0        1.067633
105              surf_8.0        1.002124
106             infra_7.0        1.194715
107              surf_6.0        0.727741
108              surf_4.0        4.483516

[109 rows x 2 columns]


In [16]:
# Définir les hyperparamètres à tester
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250, 300, 350],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}

# Recherche par grille pour trouver les meilleurs hyperparamètres
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print("Meilleurs hyperparamètres:", grid_search.best_params_)

Meilleurs hyperparamètres: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 350}


In [ ]:
# Entraîner le modèle avec les meilleurs hyperparamètres
best_rf_classifier = grid_search.best_estimator_
best_rf_classifier.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = best_rf_classifier.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Précision avec les meilleurs hyperparamètres:", accuracy)


# Afficher l'importance de chaque feature avec les meilleurs hyperparamètres
importances = best_rf_classifier.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(8, 6))
plt.title("Importance des variables dans le modèle Random Forest (avec meilleurs hyperparamètres)")
plt.bar(range(X_train.shape[1]), importances[indices], align="center")
plt.xticks(range(X_train.shape[1]), [X.columns[i] for i in indices], rotation=45)
plt.xlabel('Variables')
plt.ylabel('Importance')
plt.show()